In [1]:
import os
import tarfile

import my_sentiment_imdb_characters

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

import numpy as np
import tensorflow as tf

# Other setup
Modes = tf.estimator.ModeKeys

In [2]:
# Setup some directories
data_dir = os.path.expanduser("./t2t_data")
tmp_dir = os.path.expanduser("./tmp/t2t_datagen")

In [3]:
# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
problems.available()

['algorithmic_addition_binary40',
 'algorithmic_addition_decimal40',
 'algorithmic_cipher_shift200',
 'algorithmic_cipher_shift5',
 'algorithmic_cipher_vigenere200',
 'algorithmic_cipher_vigenere5',
 'algorithmic_identity_binary40',
 'algorithmic_identity_decimal40',
 'algorithmic_identity_vocab95_train20_eval30',
 'algorithmic_math_deepmind_all',
 'algorithmic_math_two_variables',
 'algorithmic_multiplication_binary40',
 'algorithmic_multiplication_decimal40',
 'algorithmic_reverse_binary40',
 'algorithmic_reverse_binary40_test',
 'algorithmic_reverse_decimal40',
 'algorithmic_reverse_nlplike32k',
 'algorithmic_reverse_nlplike8k',
 'algorithmic_shift_decimal40',
 'algorithmic_sort_problem',
 'audio_timit_characters_tune',
 'audio_timit_tokens8k_test',
 'audio_timit_tokens8k_tune',
 'babi_qa_concat_all_tasks_10k',
 'babi_qa_concat_all_tasks_1k',
 'babi_qa_concat_task10_10k',
 'babi_qa_concat_task10_1k',
 'babi_qa_concat_task11_10k',
 'babi_qa_concat_task11_1k',
 'babi_qa_concat_task12_

In [5]:
problem = problems.problem("my_sentiment_imdb")

# The generate_data method of a problem will download data and process it into
# a standard format ready for training and evaluation.
problem.generate_data(data_dir, tmp_dir)

INFO:tensorflow:Skipping generator because outputs files exists at ['./t2t_data/my_sentiment_imdb-unshuffled-train-00000-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00001-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00002-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00003-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00004-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00005-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00006-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00007-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00008-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00009-of-00010']


INFO:tensorflow:Skipping generator because outputs files exists at ['./t2t_data/my_sentiment_imdb-unshuffled-train-00000-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00001-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00002-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00003-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00004-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00005-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00006-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00007-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00008-of-00010', './t2t_data/my_sentiment_imdb-unshuffled-train-00009-of-00010']


INFO:tensorflow:Skipping generator because outputs files exists at ['./t2t_data/my_sentiment_imdb-unshuffled-dev-00000-of-00001']


INFO:tensorflow:Skipping generator because outputs files exists at ['./t2t_data/my_sentiment_imdb-unshuffled-dev-00000-of-00001']


INFO:tensorflow:Skipping shuffle because output files exist


INFO:tensorflow:Skipping shuffle because output files exist


In [6]:
# Get the encoders from the problem
encoders = problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(integers)

In [7]:
# Example inputs as int-tensor.
example = tfe.Iterator(problem.dataset(Modes.TRAIN, data_dir)).next()
inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.

# Example inputs as a sentence. For checking.
print(decode(inputs))
print(targets)

INFO:tensorflow:Reading data files from ./t2t_data/my_sentiment_imdb-train*


INFO:tensorflow:Reading data files from ./t2t_data/my_sentiment_imdb-train*


INFO:tensorflow:partition: 0 num_data_files: 10


INFO:tensorflow:partition: 0 num_data_files: 10


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


I first saw this movie when it originally came out. I was about 9 yrs. old and found this movie both highly entertaining and very frightening and unlike any other movie I had seen up until that time.<br /><br />BASIC PLOT: An expedition is sent out from Earth to the fourth planet of Altair, a great mainsequence star in constellation Aquilae to find out what happened to a colony of settlers which landed twenty years before and had not been heard from since.<br /><br />THEME: An inferior civilization (namely ours) comes into contact with the remains of a greatly advanced alien civilization, the Krell-200,000 years removed. The "seed" of destruction from one civilization is being passed on to another, unknowingly at first. The theme of this movie is very much Good vs. Evil.<br /><br />I first saw this movie with my brother when it came out originally. I was just a boy and the tiger scenes really did scare me as did the battle scenes with the unseen Creature-force. I was also amazed at jus

In [8]:
matching_model = [m for m in registry.list_models() if "basic" in m]
print(matching_model)

['autoencoder_basic', 'autoencoder_basic_discrete', 'basic_fc_relu', 'next_frame_basic_deterministic', 'next_frame_basic_recurrent', 'next_frame_basic_stochastic', 'next_frame_basic_stochastic_discrete']


In [9]:
matching_hparams = [h for h in registry.list_hparams() if "basic" in h]
print(matching_hparams)

['autoencoder_basic', 'autoencoder_basic_discrete', 'basic_1', 'basic_fc_small', 'basic_policy_parameters', 'evolved_transformer_tpu_basic', 'next_frame_basic_deterministic', 'next_frame_basic_recurrent', 'next_frame_basic_stochastic', 'next_frame_basic_stochastic_discrete', 'next_frame_basic_stochastic_discrete_long']


In [22]:
# Create hparams and the model
model_name = "basic_fc_relu"
hparams_set = "basic_fc_small"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="my_sentiment_imdb")
hparams.batch_size = 1

sentiment_model = registry.model(model_name)(hparams, Modes.TRAIN)

In [23]:
# Prepare for the training loop

# In Eager mode, opt.minimize must be passed a loss function wrapped with
# implicit_value_and_gradients
@tfe.implicit_value_and_gradients
def loss_fn(features):
  _, losses = sentiment_model(features)
  return losses["training"]


# Setup the training data
# BATCH_SIZE = 128

train_dataset = problem.dataset(Modes.TRAIN, data_dir)
print("start to get max length.")

max_length = 0
for example in tfe.Iterator(train_dataset):
    _tmp = example["inputs"].shape[0]
    if _tmp > max_length:
        max_length = _tmp
print(max_length)
print("Finish!")

# train_dataset = train_dataset.repeat(None).batch(BATCH_SIZE)

optimizer = tf.train.AdamOptimizer()

INFO:tensorflow:Reading data files from ./t2t_data/my_sentiment_imdb-train*


INFO:tensorflow:Reading data files from ./t2t_data/my_sentiment_imdb-train*


INFO:tensorflow:partition: 0 num_data_files: 10


INFO:tensorflow:partition: 0 num_data_files: 10


start to get max length.
3890
Finish!


In [25]:
def pad_with_zero(array):
    # Make maximum length array filled with 0.
    tmp = np.zeros(max_length, dtype=np.int64)
    
    # Paste data in max length array from beginning of array.
    tmp[:array.shape[0]] = array
    return tmp

# Train
NUM_STEPS = 100

for count, example in enumerate(tfe.Iterator(train_dataset)):
    example["inputs"] = tf.reshape(pad_with_zero(example["inputs"]), [1, -1, 1, 1])
    example["targets"] = tf.reshape(example["targets"], [1, 1, 1, 1])  # Make it 4D.
    loss, gv = loss_fn(example)
    optimizer.apply_gradients(gv)
    
    if count % 50 == 0:
        print("Step: %d, Loss: %.3f" % (count, loss.numpy()))
    if count >= NUM_STEPS:
        break

Step: 0, Loss: 0.151
Step: 50, Loss: 2.659
Step: 100, Loss: 0.153


In [30]:
sentiment_model.set_mode(Modes.EVAL)
eval_dataset = problem.dataset(Modes.EVAL, data_dir)

# Create eval metric accumulators for accuracy (ACC) and accuracy in
# top 5 (ACC_TOP5)
metrics_accum, metrics_result = metrics.create_eager_metrics(
    [metrics.Metrics.ACC, metrics.Metrics.ACC_TOP5])

for count, example in enumerate(tfe.Iterator(eval_dataset)):
    if count >= NUM_STEPS:
        break
    example["inputs"] = tf.reshape(pad_with_zero(example["inputs"]), [1, -1, 1, 1])
    example["targets"] = tf.reshape(example["targets"], [1, 1, 1, 1])  # Make it 4D.
    
    # Call the model
    predictions, _ = sentiment_model(example)
    
    # Compute and accumulate metrics
    metrics_accum(predictions, example["targets"])
    
# Print out the averaged metric values on the eval data
for name, val in metrics_result().items():
    print("%s: %.2f" % (name, val))

INFO:tensorflow:Reading data files from ./t2t_data/my_sentiment_imdb-dev*


INFO:tensorflow:Reading data files from ./t2t_data/my_sentiment_imdb-dev*


INFO:tensorflow:partition: 0 num_data_files: 1


INFO:tensorflow:partition: 0 num_data_files: 1


Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


accuracy: 0.44
accuracy_top5: 1.00
